# Complementary strain energy approach: Euler Bernoulli beam


The internal shear force and bending moment are easily computed from the equilibrium equations

\begin{align*}
T_y(x) &= F, \\
M_z(x) &= M + F(L - x)
\end{align*}

The complementary strain energy density is given by

$$
\psi^* = \int_0^\sigma \frac{\tilde{\sigma}_{xx}}{E} d\tilde{\sigma}_{xx} = \frac{\sigma_{xx}^2}{2 E} 
$$

Therefore the complementary bending energy is
$$
E_{\rm bd}^* = \frac{1}{2}\int_0^L \frac{M_z^2}{E I_z}  dx = \frac{1}{2}\int_0^L \frac{(M + F(L - x))^2}{E I_z} dx 
$$

By linearity $E_{\rm bd}^* = E_{\rm bd}$. By Castigliano theorem, it is possible to obtain the compliance matrix directly

$$ \displaystyle
\begin{pmatrix}
v_2 \\ \phi_2
\end{pmatrix} = 
\begin{pmatrix}
\frac{\partial E_{\rm bd}}{\partial F} \\
\frac{\partial E_{\rm bd}}{\partial M} 
\end{pmatrix}
$$

Developing the computations:

\begin{align*}
\frac{\partial E_{\rm bd}}{\partial F} &= \int_0^L \frac{M + F(L - x)}{E I_z}(L-x)  dx = 
\begin{bmatrix}
\frac{L^3}{3 E I_z} & \frac{L^2}{2 E I_z}
\end{bmatrix}
\begin{pmatrix}
F \\ M
\end{pmatrix} \\
\frac{\partial E_{\rm bd}}{\partial F} &= \int_0^L \frac{M + F(L - x)}{E I_z}  dx = 
\begin{bmatrix}
\frac{L^2}{2 E I_z} & \frac{L}{E I_z}
\end{bmatrix}
\begin{pmatrix}
F \\ M
\end{pmatrix}
\end{align*}

Therefore the displecement are given by 

$$
\begin{pmatrix}
v_2 \\ \phi_2
\end{pmatrix} = 
\begin{bmatrix}
\frac{L^3}{3 E I_z} & \frac{L^2}{2 E I_z} \\
\frac{L^2}{2 E I_z} & \frac{L}{E I_z}
\end{bmatrix}
\begin{pmatrix}
F \\ M
\end{pmatrix}
$$

From this relation, the compliance matrix is obtained and the stiffness matrix is simply obtained as 

$$\mathbf{K} = \mathbf{S}^{-1} = \frac{EI_z}{L^3} \begin{bmatrix}
12 & -6 L \\
-6 L & 4L^2
\end{bmatrix}$$

The calculation can be checked with the following Python code

In [19]:
import sympy as sp
from IPython.display import display, Math

M = sp.symbols('M')
F = sp.symbols('F')
L = sp.symbols('L')
EI = sp.symbols('EI')
x = sp.symbols('x')

energy_density = (M + F * (L-x))**2/(2*EI)

energy = sp.integrate(energy_density, (x, 0, L))

displacement = sp.simplify(sp.diff(energy, F))
rotation = sp.simplify(sp.diff(energy, M))

print('Displacement: ')
display(Math(sp.latex(displacement)))
print('Rotation: ')
display(Math(sp.latex(rotation)))

Displacement: 


<IPython.core.display.Math object>

Rotation: 


<IPython.core.display.Math object>

# Complementary strain energy approach: Timoshenko beam model
The Timoshenko beam model does not impose that the cross section remains perperdicular to the beam axis during rotation. Consequently, it allows accounting for the shear strain
$$
\gamma_{xy} = 2 \varepsilon_{xy} = \frac{dv}{dx} - \phi_z
$$

The shear modulus relates the shear strain and stress $\sigma_{xy} = G \gamma_{xy}$. The resultant shear force is obtained by integrating the shear stress over the cross section

$$
T_y = \int_A \sigma_{xy} dx = A G \gamma_{xy}
$$
since the strain is constant over the section. This results in a higher rigidity to what the 3D theory of elasticity would predict. This is due to the fact that the actual trend of the shear stress is not captured (for equilibrium the stress should go to zero when the lateral sides of the beam are approached). To improve the accuracy of the shear stiffness, a correction factor is introduced in the area as follows

$$
T_y = k A G \gamma_{xy}, \qquad A_{\rm eff} = k A
$$

The complementary energy for the Timoshenko beam reads

$$
E_{\rm bd+sh}^* = \int_0^L \left[\frac{(M + F(L-x))^2}{2 E I_z} + \frac{F^2}{2 G A_{\rm eff}} \right] dx
$$

According to Castigliano theorem, it follows

$$ \displaystyle
\begin{pmatrix}
v_2 \\ \phi_2
\end{pmatrix} = 
\begin{pmatrix}
\frac{\partial E_{\rm bd+sh}}{\partial F} \\
\frac{\partial E_{\rm bd+sh}}{\partial M} 
\end{pmatrix} =
\begin{bmatrix}
\left(\frac{L^3}{3EI_z} + \frac{L}{G A_{\rm eff}}  \right) & \frac{L^2}{2E I_z} \\
\frac{L^2}{2E I_z} & \frac{L}{E I_z}
\end{bmatrix}
\begin{pmatrix}
F \\ M
\end{pmatrix}
$$

The compliance matrix is $\mathbf{S} = \frac{1}{EI_z}\begin{bmatrix}
\frac{L^3}{3}\left(1 + \frac{\Psi}{4}\right) & \frac{L^2}{2} \\
\frac{L^2}{2} & L
\end{bmatrix}$

The quantity $\Psi$ defined as $\Psi := \frac{12 EI_z}{G A_{\rm eff} L^2}$ is the bending to transverse shear ratio. 

The stiffness is deduced by inversion of the compliance matrix

$$\mathbf{K} = \mathbf{S}^{-1} = \frac{EI_z}{L^3(1 +\Psi)} \begin{bmatrix}
12 & - 6 L \\
- 6 L & (4 + \Psi) L^2
\end{bmatrix}
$$


The computations can be checked via the following code:

In [23]:
import sympy as sp
from IPython.display import display, Math

M = sp.symbols('M')
F = sp.symbols('F')
L = sp.symbols('L')
EI = sp.symbols('EI')
x = sp.symbols('x')
A_eff = sp.symbols('A_{eff}')
G = sp.symbols('G')

energy_density = (M + F * (L-x))**2/(2*EI) + F**2/(2*A_eff*G)

energy = sp.integrate(energy_density, (x, 0, L))

displacement = sp.simplify(sp.diff(energy, F))
rotation = sp.simplify(sp.diff(energy, M))

print('Displacement: ')
display(Math(sp.latex(displacement)))
print('Rotation: ')
display(Math(sp.latex(rotation)))

Displacement: 


<IPython.core.display.Math object>

Rotation: 


<IPython.core.display.Math object>

# Classical costruction of the stiffness matrix for the free structure


## Euler Bernoulli beam
The stiffness matrix can be computed by expressing the  energy in terms of the finite element expansion

$$
E_{\rm bd} = \int_0^L \frac{EI_z}{2} \left(\frac{d^2 v}{d^2 x} \right)^2 dx
$$
Using the Hermite polynomials the stiffness matrix can be computed as

$$
\mathbf{K} = \int_0^L EI_z  \left(\frac{d^2 \bf{N}^\top}{d^2 x}\right)\left(\frac{d^2 \bf{N}}{d^2 x} \right) dx. 
$$
In the literature the following notation is typically used: $\displaystyle \mathbf{B} = \frac{d^2 \bf{N}}{d^2 x}$. Performing the double derivative one finds

$$
\mathbf{B} = \frac{d^2 \bf{N}}{d^2 x} = \begin{bmatrix}
-\frac{6}{L^2} + \frac{12x}{L^3} & -\frac{4}{L} + \frac{6x}{L^2} & \frac{6}{L^2} - \frac{12x}{L^3} & -\frac{2}{L} + \frac{6x}{L^2} \\
\end{bmatrix}
$$

The stiffness is therefore given by $\mathbf{K} = \int_{0}^L EI_z \mathbf{B}^\top \mathbf{B} dx$

$$
\mathbf{K} = 
\begin{bmatrix}
\frac{12 EI}{L^{3}} & \frac{6 EI}{L^{2}} & - \frac{12 EI}{L^{3}} & \frac{6 EI}{L^{2}} \\
\frac{6 EI}{L^{2}} & \frac{4 EI}{L} & - \frac{6 EI}{L^{2}} & \frac{2 EI}{L} \\
- \frac{12 EI}{L^{3}} & - \frac{6 EI}{L^{2}} & \frac{12 EI}{L^{3}} & - \frac{6 EI}{L^{2}} \\
\frac{6 EI}{L^{2}} & \frac{2 EI}{L} & - \frac{6 EI}{L^{2}} & \frac{4 EI}{L}
\end{bmatrix}
$$
This python code computes the stiffness matrix 


In [17]:
import sympy as sp

EI = sp.symbols('EI')
L = sp.symbols('L')
x = sp.symbols('x')

B_1 = - 6/L**2 + 12*x/L**3
B_2 = - 4/L + 6*x/L**2
B_3 = + 6/L**2 - 12*x/L**3
B_4 = - 2/L + 6*x/L**2

B_vec = sp.Matrix([[B_1, B_2, B_3, B_4]])

BT = sp.transpose(B_vec)
BTB = BT * B_vec
K = sp.sympify(EI * sp.integrate(BTB, (x, 0, L)))

K

'\\left[\\begin{matrix}\\frac{12 EI}{L^{3}} & \\frac{6 EI}{L^{2}} & - \\frac{12 EI}{L^{3}} & \\frac{6 EI}{L^{2}}\\\\\\frac{6 EI}{L^{2}} & \\frac{4 EI}{L} & - \\frac{6 EI}{L^{2}} & \\frac{2 EI}{L}\\\\- \\frac{12 EI}{L^{3}} & - \\frac{6 EI}{L^{2}} & \\frac{12 EI}{L^{3}} & - \\frac{6 EI}{L^{2}}\\\\\\frac{6 EI}{L^{2}} & \\frac{2 EI}{L} & - \\frac{6 EI}{L^{2}} & \\frac{4 EI}{L}\\end{matrix}\\right]'

## Complement: Timoshenko beam

For the Timoshenko beam an additional variable needs to be considered, i.e. the rotation of the cross section. The interpolation functions for the rotations, are taken to be the derivative of the Hermite polynomials (modulo a scaling factor). For the vertical displacement the interpolation functions are taken to be the Hermite polynomials plus a correction.

$$
v(x) = \mathbf{N}_v \mathbf{v}, \qquad
\phi(x) = \mathbf{N}_\phi\mathbf{v}
$$
The interpolation functions for $v$ are given by
$$
\mathbf{N}_v^\top = \begin{bmatrix}
\mathbf{N}_{1v} \\
\mathbf{N}_{2v} \\
\mathbf{N}_{3v} \\
\mathbf{N}_{4v} \\
\end{bmatrix} = \begin{bmatrix}
\frac{2{N}_{1} + \Psi(1 - x/L)}{2(1 + \Psi)} \\
\frac{2{N}_{2} + \Psi x(1 - x/L)}{2(1 + \Psi)} \\
\frac{2{N}_{3} + \Psi x/L}{2(1 + \Psi)} \\
\frac{2{N}_{4} - \Psi x(1 - x/L)}{2(1 + \Psi)} \\
\end{bmatrix} 
$$phi
where $N_i, \; i \in \{1, 2, 3, 4\}$ are the Hermite polynomials. For the rotation the intepolation functions are given by

$$
\mathbf{N}_\phi^\top = \begin{bmatrix}
\mathbf{N}_{1\phi} \\
\mathbf{N}_{2\phi} \\
\mathbf{N}_{3\phi} \\
\mathbf{N}_{4\phi} \\
\end{bmatrix} = \begin{bmatrix}
(1 + \Psi)^{-1} \frac{dN_1}{dx} \\
L (1 + \Psi)^{-1} (\frac{dN_2}{dx} + \Psi(1 - x/L)) \\
(1 + \Psi)^{-1} \frac{dN_3}{dx} \\
L (1 + \Psi)^{-1} (\frac{dN_2}{dx} + \Psi x/L) \\
\end{bmatrix} 
$$

Now the energy (that includes both bending and shear) is expressed in terms of the displacement and the rotation

$$
E(v,\phi) = \int_{0}^{L} \left[ \frac{EI_z}{2} \left(\frac{d\phi}{dx} \right)^2 + \frac{GA_{\rm eff}}{2} \left(\frac{dv}{dx} - \phi\right)^2  \right] dx
$$

After some algebraic manipulations the stiffness matrix is obtained

$$
\mathbf{K}_{T} = \frac{EI_z}{L^3 (1 + \Psi)} \begin{bmatrix}
12 & 6L & -12 & 6L \\
 & (4 + \Psi) L^2 & -6L & (2 - \Psi) L^2 \\
 & & 12 & -6L \\
 \rm{sym} & & & (4 + \Psi) L^2
\end{bmatrix}
$$